In [ ]:
pip install strawberryfields

In [3]:
!python --version


Python 3.10.12


In [4]:
pip install version_check

ERROR: Could not find a version that satisfies the requirement version_check (from versions: none)
ERROR: No matching distribution found for version_check


In [5]:
import numpy as np
import os

import tensorflow as tf

import strawberryfields as sf
from strawberryfields.ops import Dgate, BSgate, Kgate, Sgate, Rgate

import sys
sys.path.append("..")
# import version_check

In [6]:
import csv
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import csv


# Data Processing

In [7]:
# Install the Kaggle library
!pip install kaggle

# Upload kaggle.json file
from google.colab import files
files.upload()  # This will prompt you to upload kaggle.json

# Make a directory for Kaggle and move the file there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Now you can download the dataset
!kaggle datasets download -d mlg-ulb/creditcardfraud -p /content/datasets --unzip

print("Dataset downloaded to: /content/datasets")

finance_df=pd.read_csv("/content/datasets/creditcard.csv",delimiter=',')

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
 99% 65.0M/66.0M [00:02<00:00, 33.6MB/s]
100% 66.0M/66.0M [00:02<00:00, 23.5MB/s]
Dataset downloaded to: /content/datasets


In [8]:
finance_df=pd.read_csv("/content/datasets/creditcard.csv",delimiter=',')

In [9]:
with open('/content/datasets/creditcard.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    data = list(csv_reader)

# If you want to compare or transform, you can print or manipulate both `finance_df` and `data`
print("DataFrame loaded using pandas:")
print(finance_df.head())

print("\nData loaded using csv.reader:")
print(data[:5])  # Display the first 5 rows of the list


DataFrame loaded using pandas:
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

 

In [10]:
data = data[1:]
data_genuine = []
data_fraudulent = []

# Splitting genuine and fraudulent data
for i in range(len(data)):
    if int(data[i][30]) == 0:
        data_genuine.append([float(i) for i in data[i]])
    if int(data[i][30]) == 1:
        data_fraudulent.append([float(i) for i in data[i]])

fraudulent_data_points = len(data_fraudulent)

# We want the genuine data points to be 3x the fraudulent ones
undersampling_ratio = 3

genuine_data_points = fraudulent_data_points * undersampling_ratio

random.shuffle(data_genuine)
random.shuffle(data_fraudulent)

# Fraudulent and genuine transactions are split into two datasets for cross validation

data_fraudulent_1 = data_fraudulent[:int(fraudulent_data_points / 2)]
data_fraudulent_2 = data_fraudulent[int(fraudulent_data_points / 2):]

data_genuine_1 = data_genuine[:int(genuine_data_points / 2)]
data_genuine_2 = data_genuine[int(genuine_data_points / 2):genuine_data_points]
data_genuine_remaining = data_genuine[genuine_data_points:]

random.shuffle(data_fraudulent_1)
random.shuffle(data_fraudulent_2)
random.shuffle(data_genuine_1)
random.shuffle(data_genuine_2)

data_genuine=data_genuine_1+data_genuine_2
data_fraudulent=data_fraudulent_1+data_fraudulent_2
# np.savetxt('creditcard_genuine_1.csv', data_genuine_1, delimiter=',')
# np.savetxt('creditcard_genuine_2.csv', data_genuine_2, delimiter=',')
# np.savetxt('creditcard_fraudulent_1.csv', data_fraudulent_1, delimiter=',')
# np.savetxt('creditcard_fraudulent_2.csv', data_fraudulent_2, delimiter=',')
# # Larger datasets are used for testing, including genuine transactions unseen in training
# np.savetxt('creditcard_combined_1_big.csv', data_fraudulent_1 + data_genuine_1 + data_genuine_remaining, delimiter=',')
# np.savetxt('creditcard_combined_2_big.csv', data_fraudulent_2 + data_genuine_2 + data_genuine_remaining, delimiter=',')

In [11]:
print(len(data_genuine))  # Number of elements in the list
print(len(data_fraudulent))  # Number of elements in the list


1476
492


In [12]:
# # Data outputted from data_processor.py
# data_genuine = np.loadtxt('creditcard_genuine_1.csv', delimiter=',')
# data_fraudulent = np.loadtxt('creditcard_fraudulent_1.csv', delimiter=',')

# Combining genuine and fraudulent data
data_combined = np.append(data_genuine, data_fraudulent, axis=0)
data_points = len(data_combined)

In [13]:
data_combined.shape

(1968, 31)

# Model Architecture

In [30]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [31]:
# Input neurons
input_neurons = 10
# Widths of hidden layers
nn_architecture = [10, 10]
# Output neurons of classical part
output_neurons = 14

# Defining classical network parameters
input_classical_layer = tf.placeholder(tf.float32, shape=[batch_size, input_neurons])

layer_matrix_1 = tf.Variable(tf.random_normal(shape=[input_neurons, nn_architecture[0]]))
offset_1 = tf.Variable(tf.random_normal(shape=[nn_architecture[0]]))

layer_matrix_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[0], nn_architecture[1]]))
offset_2 = tf.Variable(tf.random_normal(shape=[nn_architecture[1]]))

layer_matrix_3 = tf.Variable(tf.random_normal(shape=[nn_architecture[1], output_neurons]))
offset_3 = tf.Variable(tf.random_normal(shape=[output_neurons]))

# Creating hidden layers and output
layer_1 = tf.nn.elu(tf.matmul(input_classical_layer, layer_matrix_1) + offset_1)
layer_2 = tf.nn.elu(tf.matmul(layer_1, layer_matrix_2) + offset_2)

output_layer = tf.nn.elu(tf.matmul(layer_2, layer_matrix_3) + offset_3)

In [19]:

# Two modes required: one for "genuine" transactions and one for "fradulent"
mode_number = 2
# Number of photonic quantum layers
depth = 4

# Fock basis truncation
cutoff = 10
# Number of batches in optimization
reps = 30000

# Label for simulation
simulation_label = 1

# Number of batches to use in the optimization
batch_size = 24

# Random initialization of gate parameters
sdev_photon = 0.1
sdev = 1

# Variable clipping values
disp_clip = 5
sq_clip = 5
kerr_clip = 1

# If loading from checkpoint, previous batch number reached
ckpt_val = 0

# Number of repetitions between each output to TensorBoard
tb_reps = 100
# Number of repetitions between each model save
savr_reps = 1000

model_string = str(simulation_label)

# Target location of output
folder_locator = './outputs/'

# Locations of TensorBoard and model save outputs
board_string = folder_locator + 'tensorboard/' + model_string + '/'
checkpoint_string = folder_locator + 'models/' + model_string + '/'

#                                   Setting up the classical NN input


AttributeError: module 'tensorflow' has no attribute 'placeholder'

#                                   Defining QNN parameters

In [32]:
# ===================================================================================
#                                   Defining QNN parameters
# ===================================================================================

# Number of beamsplitters in interferometer
bs_in_interferometer = int(1.0 * mode_number * (mode_number - 1) / 2)

with tf.name_scope('variables'):
    bs_variables = tf.Variable(tf.random_normal(shape=[depth, bs_in_interferometer, 2, 2]
                                                , stddev=sdev))
    phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number, 2], stddev=sdev))

    sq_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                          , stddev=sdev_photon))
    sq_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                      , stddev=sdev))
    disp_magnitude_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                            , stddev=sdev_photon))
    disp_phase_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number]
                                                        , stddev=sdev))
    kerr_variables = tf.Variable(tf.random_normal(shape=[depth, mode_number], stddev=sdev_photon))

parameters = [layer_matrix_1, offset_1, layer_matrix_2, offset_2, layer_matrix_3, offset_3, bs_variables,
              phase_variables, sq_magnitude_variables, sq_phase_variables, disp_magnitude_variables,
              disp_phase_variables, kerr_variables]



#                                   Constructing quantum layers

In [33]:
# ===================================================================================
#                                   Constructing quantum layers
# ===================================================================================


# Defining input QNN layer, whose parameters are set by the outputs of the classical network
def input_qnn_layer():
    with tf.name_scope('inputlayer'):
        Sgate(tf.clip_by_value(output_layer[:, 0], -sq_clip, sq_clip), output_layer[:, 1]) | q[0]
        Sgate(tf.clip_by_value(output_layer[:, 2], -sq_clip, sq_clip), output_layer[:, 3]) | q[1]

        BSgate(output_layer[:, 4], output_layer[:, 5]) | (q[0], q[1])

        Rgate(output_layer[:, 6]) | q[0]
        Rgate(output_layer[:, 7]) | q[1]

        Dgate(tf.clip_by_value(output_layer[:, 8], -disp_clip, disp_clip), output_layer[:, 9]) \
        | q[0]
        Dgate(tf.clip_by_value(output_layer[:, 10], -disp_clip, disp_clip), output_layer[:, 11]) \
        | q[1]

        Kgate(tf.clip_by_value(output_layer[:, 12], -kerr_clip, kerr_clip)) | q[0]
        Kgate(tf.clip_by_value(output_layer[:, 13], -kerr_clip, kerr_clip)) | q[1]


# Defining standard QNN layers
def qnn_layer(layer_number):
    with tf.name_scope('layer_{}'.format(layer_number)):
        BSgate(bs_variables[layer_number, 0, 0, 0], bs_variables[layer_number, 0, 0, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 0]) | q[i]

        for i in range(mode_number):
            Sgate(tf.clip_by_value(sq_magnitude_variables[layer_number, i], -sq_clip, sq_clip),
                  sq_phase_variables[layer_number, i]) | q[i]

        BSgate(bs_variables[layer_number, 0, 1, 0], bs_variables[layer_number, 0, 1, 1]) \
        | (q[0], q[1])

        for i in range(mode_number):
            Rgate(phase_variables[layer_number, i, 1]) | q[i]

        for i in range(mode_number):
            Dgate(tf.clip_by_value(disp_magnitude_variables[layer_number, i], -disp_clip,
                                   disp_clip), disp_phase_variables[layer_number, i]) | q[i]

        for i in range(mode_number):
            Kgate(tf.clip_by_value(kerr_variables[layer_number, i], -kerr_clip, kerr_clip)) | q[i]


In [40]:
prog = sf.Program(mode_number)

# construct the circuit
with prog.context as q:

  input_qnn_layer()

  for i in range(depth):
    qnn_layer(i)

# run the engine
eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 5})
state = eng.run(prog)

ValueError: The custom_gradient decorator currently supports keywords arguments only when eager execution is enabled.

#                                   Defining QNN

# Personal Try

In [24]:
import tensorflow as tf
import strawberryfields as sf
from strawberryfields.ops import Sgate, BSgate, Rgate, Dgate, Kgate

# Function to create the classical model
def create_classical_model(input_neurons):
    """Creates and returns a classical neural network model."""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(14, activation='elu', input_shape=(input_neurons,))
    ])
    return model

def create_quantum_network(depth, mode_number, output_layer,
                           bs_variables, phase_variables,
                           sq_magnitude_variables, sq_phase_variables,
                           disp_magnitude_variables, disp_phase_variables,
                           kerr_variables):
    """Creates and returns a quantum program (QNN)."""
    prog = sf.Program(mode_number)

    with prog.context as q:  # Explicitly create `q` within the context
        # Input QNN Layer
        input_qnn_layer(output_layer, q)

        # QNN Layers
        for i in range(depth):
            qnn_layer(i, q, bs_variables, phase_variables, sq_magnitude_variables,
                      sq_phase_variables, disp_magnitude_variables, disp_phase_variables,
                      kerr_variables)

    return prog

# Function to run the quantum network
def run_quantum_network(prog, cutoff, batch_size):
    """Runs the quantum program (QNN) and returns the results."""
    eng = sf.Engine("tf", backend_options={"cutoff_dim": cutoff, "batch_size": batch_size})

    try:
        # Run the program
        result = eng.run(prog, eval=False)

        # Extract the state
        state = result.state

        if hasattr(state, 'ket'):
            ket = state.ket()
            print("Ket retrieved successfully. Shape:", ket.shape)
            return ket
        elif hasattr(state, 'dm'):
            dm = state.dm()
            print("Density matrix retrieved successfully. Shape:", dm.shape)
            return dm
        else:
            print("No state representation available.")
            return None
    except Exception as e:
        print("Error during quantum circuit execution:", e)
        return None



In [25]:
data_points

1968

In [28]:
depth = 3
mode_number = 2
batch_size = 4
cutoff = 10
input_neurons = 10

# Create the classical neural network
model = create_classical_model(input_neurons)
# Ensure data_combined is a TensorFlow tensor
if not isinstance(data_combined, tf.Tensor):
    data_combined = tf.convert_to_tensor(data_combined, dtype=tf.float32)

# Verify the shape of data_combined
print("Shape of data_combined:", data_combined.shape)

# Print classical NN output to verify
print("Classical NN Output Shape:", output_layer.shape)
print("Classical NN Output Sample:", output_layer[0])

Shape of data_combined: (1968, 31)
Classical NN Output Shape: (None, 14)
Classical NN Output Sample: <KerasTensor shape=(14,), dtype=float32, sparse=False, name=keras_tensor_9>


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


In [29]:
bs_variables = np.random.random((depth, 1, 2, 2))  # Beam splitter parameters
phase_variables = np.random.random((depth, mode_number, 2))  # Phase parameters
sq_magnitude_variables = np.random.random((depth, mode_number))  # Squeezing magnitudes
sq_phase_variables = np.random.random((depth, mode_number))  # Squeezing phases
disp_magnitude_variables = np.random.random((depth, mode_number))  # Displacement magnitudes
disp_phase_variables = np.random.random((depth, mode_number))  # Displacement phases
kerr_variables = np.random.random((depth, mode_number))  # Kerr interaction parameters

# Create the quantum network
prog = create_quantum_network(depth, mode_number, output_layer,
                              bs_variables, phase_variables,
                              sq_magnitude_variables, sq_phase_variables,
                              disp_magnitude_variables, disp_phase_variables,
                              kerr_variables)

# Run the quantum network
results = run_quantum_network(prog, cutoff, batch_size)


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


# Training

In [ ]:
def classify_data(model, data, prog, depth, cutoff, batch_size,
                  mode_number, bs_variables, phase_variables,
                  sq_magnitude_variables, sq_phase_variables,
                  disp_magnitude_variables, disp_phase_variables,
                  kerr_variables):
    """
    Perform classification using the QNN pipeline.

    Args:
        model: The classical NN model.
        data: Input data for classification.
        prog: The quantum program (QNN).
        depth, cutoff, batch_size, mode_number: Quantum parameters.
        bs_variables, phase_variables, sq_magnitude_variables,
        sq_phase_variables, disp_magnitude_variables, disp_phase_variables,
        kerr_variables: Quantum parameters for the QNN.

    Returns:
        Predicted classes for the input data.
    """
    # Step 1: Encode data with the classical NN
    output_layer = model(data)  # Output from the classical NN

    # Step 2: Create and run the quantum network
    prog = create_quantum_network(depth, mode_number, output_layer,
                                  bs_variables, phase_variables,
                                  sq_magnitude_variables, sq_phase_variables,
                                  disp_magnitude_variables, disp_phase_variables,
                                  kerr_variables)
    ket = run_quantum_network(prog, cutoff, batch_size)

    # print(ket)

    # Step 3: Extract classification-relevant information
    # Example: Use photon probabilities in the Fock basis
    probabilities = tf.math.abs(ket) ** 2  # Compute probabilities

    print(probabilities)

    # For example, sum probabilities over modes to get class scores
    class_scores = tf.reduce_sum(probabilities, axis=(1, 2))

    print(class_scores)
    # Step 4: Decision rule for classification
    predictions = tf.argmax(class_scores, axis=-1).numpy()  # Class with the highest score

    return predictions


In [1]:
# Input data for classification
test_data = tf.random.normal([batch_size, input_neurons])  # Dummy input data

# Call the classification function
predictions = classify_data(model, test_data, prog, depth, cutoff, batch_size,
                             mode_number, bs_variables, phase_variables,
                             sq_magnitude_variables, sq_phase_variables,
                             disp_magnitude_variables, disp_phase_variables,
                             kerr_variables)

# print("Predicted Classes:", predictions)


NameError: name 'tf' is not defined

# From Github Implementation

In [ ]:
import tensorflow as tf
import numpy as np

# Define parameters (example, replace with actual implementation)
batch_size = 32
input_neurons = 10
parameters = []  # Replace with actual trainable variables
checkpoint_string = './checkpoints/'  # Replace with your checkpoint directory
reps = 1000
savr_reps = 100
tb_reps = 10
board_string = './logs/'  # TensorBoard logs directory
ckpt_val = 0

def func_to_minimise(input_data, one_hot_input):
    # Ensure input_data is cast to float32 for compatibility
    input_data = tf.cast(input_data, tf.float32)
    return tf.reduce_mean(tf.square(input_data - tf.cast(one_hot_input, tf.float32)))


@tf.function
def train_step(input_data, one_hot_input):
    with tf.GradientTape() as tape:
        cost_func_value = func_to_minimise(input_data, one_hot_input)
    gradients = tape.gradient(cost_func_value, parameters)
    optimiser.apply_gradients(zip(gradients, parameters))
    return cost_func_value



# Optimizer
optimiser = tf.keras.optimizers.Adam()

# Checkpointing
ckpt = tf.train.Checkpoint(optimizer=optimiser, model_parameters=parameters)
manager = tf.train.CheckpointManager(ckpt, directory=checkpoint_string, max_to_keep=5)

if ckpt_val != 0:
    ckpt.restore(manager.latest_checkpoint).expect_partial()
    print(f"Restored checkpoint from {manager.latest_checkpoint}")

# TensorBoard writer
summary_writer = tf.summary.create_file_writer(board_string)

counter = ckpt_val
opt_val = 1e20
opt_position = 0
new_opt = False

while counter <= reps:
    # Simulated data, replace with actual data loading and processing
    data_combined = np.random.random((batch_size * 10, input_neurons + 2))  # Example dataset
    np.random.shuffle(data_combined)
    split_data = np.split(data_combined, len(data_combined) // batch_size)

    for batch in split_data:
        if counter > reps:
            break

        # Input and classification
        data_points_principal_components = batch[:, 1:3]  # Use 2 features to match `one_hot_input`

        classes = batch[:, -1]
        one_hot_input = np.zeros((batch_size, 2))

        for i in range(batch_size):
            if int(classes[i]) == 0:
                one_hot_input[i] = [1, 0]
            else:
                one_hot_input[i] = [0, 1]

        # Training step
        cost_func_value = train_step(data_points_principal_components, one_hot_input)

        if counter % tb_reps == 0:
            with summary_writer.as_default():
                tf.summary.scalar("Cost", cost_func_value, step=counter)

        # Update optimal value
        if cost_func_value < opt_val:
            opt_val = cost_func_value
            opt_position = counter
            new_opt = True

        # Save model at intervals
        if (counter % savr_reps == 0) and new_opt:
            manager.save(checkpoint_number=counter)
            np.savetxt(checkpoint_string + 'optimum.txt', [opt_position, opt_val])

        counter += 1


ValueError: in user code:

    File "<ipython-input-56-f11e2cac72a7>", line 26, in train_step  *
        optimiser.apply_gradients(zip(gradients, parameters))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py", line 290, in apply_gradients  **
        grads, trainable_variables = zip(*grads_and_vars)

    ValueError: not enough values to unpack (expected 2, got 0)
